In [ ]:
import tensorflow as tf
import json
import numpy as np

## Load training, validation and test sets

In [ ]:
with open('train_set.json') as f:
    train_data = np.array(json.load(f)['target'])
with open('test_set.json') as f:
    test_data = np.array(json.load(f)['target'])
with open('validate_set.json') as f:
    validation_data = np.array(json.load(f)['target'])

In [ ]:
history_length = 720 # one month of hourly data = 30 * 24
output_length = 168 # one week of hourly data = 7 * 24
step = 24 # one day

In [ ]:
def reshape_data(data, input_length, output_length):
    features = []
    output = []
    for i in range(0, len(data) - input_length - output_length):
        features.append(data[i:i+input_length])
        output.append(data[i+input_length:i+input_length+output_length])
    return np.array(features), np.array(output)

In [ ]:
x, y = reshape_data(train_data, history_length, output_length)
x_val, y_val = reshape_data(validation_data, history_length, output_length)

## Create the model

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(x.shape[1],), name="previous_month"))
model.add(tf.keras.layers.Reshape((x.shape[1],1)))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dense(168))

model.compile(optimizer='adam', loss='mae')

In [ ]:
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
history = model.fit(x, y,
                    batch_size=500,
                    callbacks=[callback],
                    epochs=2000,
                    validation_data=(x_val, y_val))

# Save in SavedModel format

In [ ]:
save_dir = 'saved_model/model'
model.save(save_dir)

# Upload to Waylay platform

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from zipfile import ZipFile
import os
import shutil
from io import BytesIO

In [ ]:
def upload_model(model_name, directory, upload_url, api_key, api_secret):
    # create zip
    shutil.make_archive('model', 'zip', directory)
    with open('model.zip', 'rb') as f:
        upload_file = BytesIO(f.read())
        
    # upload to waylay
    resp = requests.post(upload_url,
                         files={"file": ('model.zip', upload_file)},
                         data={"name": model_name, "description": description, "framework": "tensorflow" },
                         auth=HTTPBasicAuth(api_key, api_secret))

    if resp.status_code == 200:
        return 'Succes'
    else:
        return 'Failed'

In [ ]:
# https://docs.waylay.io/api/rest/#authentication
api_key = 'Replace with your apiKey'
api_secret = 'Replace with your apiSecret'
byoml_url = 'Replace with the correct url'
print(upload_model('replace with a model_name', save_dir, byoml_url, api_key, api_secret))